![NYC Skyline](nyc.jpg)

Welcome to New York City, one of the most-visited cities in the world. There are many Airbnb listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this project, we will take a closer look at the New York Airbnb market by combining data from multiple file types like `.csv`, `.tsv`, and `.xlsx`.

Recall that **CSV**, **TSV**, and **Excel** files are three common formats for storing data. 
Three files containing data on 2019 Airbnb listings are available to you:

**data/airbnb_price.csv**
This is a CSV file containing data on Airbnb listing prices and locations.
- **`listing_id`**: unique identifier of listing
- **`price`**: nightly listing price in USD
- **`nbhood_full`**: name of borough and neighborhood where listing is located

**data/airbnb_room_type.xlsx**
This is an Excel file containing data on Airbnb listing descriptions and room types.
- **`listing_id`**: unique identifier of listing
- **`description`**: listing description
- **`room_type`**: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**data/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: unique identifier of listing
- **`host_name`**: name of listing host
- **`last_review`**: date when the listing was last reviewed

## GOAL
Find answers to the following questions:
<ul>
    <li>What are the dates of the earliest and most recent reviews? </li>
    <li>How many of the listings are private rooms? </li>
    <li> What is the average listing price rounded to the nearest two decimal places?</li>
    <li>Combine the answers into one DataFrame called review_dates with four columns in the following order: first_reviewed, last_reviewed, nb_private_rooms, and avg_price. The DataFrame should only contain one row of values.</li>
</ul>

In [212]:
# Import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


### Loading dataset 

In [213]:
# loading price dataset 
price = pd.read_csv('data/airbnb_price.csv')
print(price.info())
price.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25209 non-null  int64 
 1   price        25209 non-null  object
 2   nbhood_full  25209 non-null  object
dtypes: int64(1), object(2)
memory usage: 591.0+ KB
None


,listing_id,price,nbhood_full
0,2595,225 dollars,"Manhattan, Midtown"
1,3831,89 dollars,"Brooklyn, Clinton Hill"
2,5099,200 dollars,"Manhattan, Murray Hill"
3,5178,79 dollars,"Manhattan, Hell's Kitchen"
4,5238,150 dollars,"Manhattan, Chinatown"


In [214]:
# loading room_type dataset
room_type = pd.read_excel('data/airbnb_room_type.xlsx')
print(room_type.info())
room_type.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25209 non-null  int64 
 1   description  25199 non-null  object
 2   room_type    25209 non-null  object
dtypes: int64(1), object(2)
memory usage: 591.0+ KB
None


,listing_id,description,room_type
0,2595,Skylit Midtown Castle,Entire home/apt
1,3831,Cozy Entire Floor of Brownstone,Entire home/apt
2,5099,Large Cozy 1 BR Apartment In Midtown East,Entire home/apt
3,5178,Large Furnished Room Near B'way,private room
4,5238,Cute & Cozy Lower East Side 1 bdrm,Entire home/apt


In [215]:
# loading last_review dataset
last_review = pd.read_csv(
    'data/airbnb_last_review.tsv',
    sep='\t',
    parse_dates=['last_review']
)
print(last_review.info())
last_review['last_review'] = last_review['last_review'].dt.strftime('%Y-%m-%d')
last_review.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   listing_id   25209 non-null  int64         
 1   host_name    25201 non-null  object        
 2   last_review  25209 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 591.0+ KB
None


,listing_id,host_name,last_review
0,2595,Jennifer,2019-05-21
1,3831,LisaRoxanne,2019-07-05
2,5099,Chris,2019-06-22
3,5178,Shunichi,2019-06-24
4,5238,Ben,2019-06-09


Now, we should join these different dataframes on listing_id column and make one, common dataframe "airbnb". First, let's join last_review and room_type and then join their joint to price.

In [216]:
# join price and room_type
review_room = pd.merge(last_review, room_type, how='left', on='listing_id')
# join price_room and last_review
airbnb = pd.merge(review_room, price, how='left', on='listing_id')
airbnb.info()
airbnb.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25209 entries, 0 to 25208
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25209 non-null  int64 
 1   host_name    25201 non-null  object
 2   last_review  25209 non-null  object
 3   description  25199 non-null  object
 4   room_type    25209 non-null  object
 5   price        25209 non-null  object
 6   nbhood_full  25209 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.5+ MB


,listing_id,host_name,last_review,description,room_type,price,nbhood_full
0,2595,Jennifer,2019-05-21,Skylit Midtown Castle,Entire home/apt,225 dollars,"Manhattan, Midtown"
1,3831,LisaRoxanne,2019-07-05,Cozy Entire Floor of Brownstone,Entire home/apt,89 dollars,"Brooklyn, Clinton Hill"
2,5099,Chris,2019-06-22,Large Cozy 1 BR Apartment In Midtown East,Entire home/apt,200 dollars,"Manhattan, Murray Hill"
3,5178,Shunichi,2019-06-24,Large Furnished Room Near B'way,private room,79 dollars,"Manhattan, Hell's Kitchen"
4,5238,Ben,2019-06-09,Cute & Cozy Lower East Side 1 bdrm,Entire home/apt,150 dollars,"Manhattan, Chinatown"


### Data cleaning

Let us now explore if airbnb has any null values

In [217]:
airbnb.isnull().sum()

listing_id      0
host_name       8
last_review     0
description    10
room_type       0
price           0
nbhood_full     0
dtype: int64

We can see that only the description and host_name columns have some null values. Let's see if there is any particular reason for them.

In [218]:
airbnb_null = airbnb[airbnb.isnull().any(axis=1)]
airbnb_null

,listing_id,host_name,last_review,description,room_type,price,nbhood_full
263,100184,NaN,2019-07-08,Bienvenue,private room,50 dollars,"Queens, Queens Village"
3040,4183989,NaN,2019-05-23,SPRING in the City!! Zen-Style Tranquil Bedroom,Private room,86 dollars,"Manhattan, Harlem"
4039,6292866,NaN,2019-06-19,Modern Quiet Gem Near All,entire home/apt,85 dollars,"Brooklyn, East Flatbush"
4269,6786181,NaN,2019-06-19,R&S Modern Spacious Hideaway,Entire home/apt,100 dollars,"Brooklyn, East Flatbush"
4688,7851219,John,2019-06-18,NaN,PRIVATE ROOM,60 dollars,"Brooklyn, Williamsburg"
4690,7854307,John,2019-06-15,NaN,private room,60 dollars,"Brooklyn, Williamsburg"
4696,7858673,John,2019-06-08,NaN,Private room,60 dollars,"Brooklyn, Williamsburg"
4707,7873655,John,2019-06-21,NaN,PRIVATE ROOM,60 dollars,"Brooklyn, Williamsburg"
4709,7886635,John,2019-06-18,NaN,Private room,60 dollars,"Brooklyn, Williamsburg"
4717,7901635,John,2019-06-23,NaN,Private room,60 dollars,"Brooklyn, Williamsburg"


Most of the missing values are for 'private room' room_type and are last_review dates are from June 2019. But other than that, there doesn't seem to much of a pattern here. As these are text columns, any imputation method is meaningless. The best approach, it seems is to drop the null values

In [219]:
airbnb = airbnb.dropna()
airbnb.isnull().sum()

Another issue that can be observed is that "room_type" column has practically only two unique categories -- 'entire home/apt', 'shared room' 'private room' but the casing of letter varies greatly. This should be fixed by making all of them as lower case letters and storing them as categories.

In [220]:
airbnb['room_type'].unique()

array(['Entire home/apt', 'private room', 'Private room',
       'entire home/apt', 'PRIVATE ROOM', 'shared room',
       'ENTIRE HOME/APT', 'Shared room', 'SHARED ROOM'], dtype=object)

In [221]:
airbnb['room_type'] = airbnb['room_type'].str.lower()
airbnb['room_type'].unique()

array(['entire home/apt', 'private room', 'shared room'], dtype=object)

The 'price' column is of dtype object when it should rather be an integer for easier calculation. For that purpose, it must be stripped of " dollars". The column can be renamed to 'price (in dollars)' and it serves the same purpose.

In [222]:
airbnb['price'].dtype

dtype('O')

In [223]:
airbnb['price'] = airbnb['price'].str.strip(" dollars")
print(airbnb['price'].dtype)
airbnb['price'] = airbnb['price'].astype("int")
print(airbnb['price'].dtype)

object
int64


In [224]:
airbnb = airbnb.rename(columns={'price':'price (in dollars)'})
airbnb.head()

,listing_id,host_name,last_review,description,room_type,price (in dollars),nbhood_full
0,2595,Jennifer,2019-05-21,Skylit Midtown Castle,entire home/apt,225,"Manhattan, Midtown"
1,3831,LisaRoxanne,2019-07-05,Cozy Entire Floor of Brownstone,entire home/apt,89,"Brooklyn, Clinton Hill"
2,5099,Chris,2019-06-22,Large Cozy 1 BR Apartment In Midtown East,entire home/apt,200,"Manhattan, Murray Hill"
3,5178,Shunichi,2019-06-24,Large Furnished Room Near B'way,private room,79,"Manhattan, Hell's Kitchen"
4,5238,Ben,2019-06-09,Cute & Cozy Lower East Side 1 bdrm,entire home/apt,150,"Manhattan, Chinatown"


The 'last_review' column which contains the date on which the last review was posted is of dtype=object. This must be converted to datetime type, otherwise the analysis would be problematic.

In [225]:
print(airbnb['last_review'].dtype)
airbnb['last_review'] = pd.to_datetime(airbnb['last_review']).dt.normalize()
print(airbnb['last_review'].dtype)

object
datetime64[ns]


In [226]:
airbnb.head()

,listing_id,host_name,last_review,description,room_type,price (in dollars),nbhood_full
0,2595,Jennifer,2019-05-21,Skylit Midtown Castle,entire home/apt,225,"Manhattan, Midtown"
1,3831,LisaRoxanne,2019-07-05,Cozy Entire Floor of Brownstone,entire home/apt,89,"Brooklyn, Clinton Hill"
2,5099,Chris,2019-06-22,Large Cozy 1 BR Apartment In Midtown East,entire home/apt,200,"Manhattan, Murray Hill"
3,5178,Shunichi,2019-06-24,Large Furnished Room Near B'way,private room,79,"Manhattan, Hell's Kitchen"
4,5238,Ben,2019-06-09,Cute & Cozy Lower East Side 1 bdrm,entire home/apt,150,"Manhattan, Chinatown"


Now, we can proceed to our data analysis

### Earliest and most recent reviews

In [227]:
import datetime as dt
earliest = min(airbnb['last_review']).date
most_recent = max(airbnb['last_review']).date
print(f'Earliest review was on {earliest} and the most recent review was on {most_recent}.')

Earliest review was on 2019-01-01 00:00:00 and the most recent review was on 2019-07-09 00:00:00.


### Distribution of listings

In [228]:
prvt_room_count = (airbnb['room_type'] == 'private room').sum()
prvt_room_count

11356

### Average listing price

In [229]:
avg_listing_price = airbnb['price (in dollars)'].mean().round(2)
avg_listing_price

141.78

### combining the results

In [230]:
columns = {
    'first_reviewed': earliest,
    'last_reviewed': most_recent,
    'nb_private_rooms': prvt_room_count,
    'avg_price': avg_listing_price
}
# combining it all in a dataframe
review_dates = pd.DataFrame([columns])
review_dates[['first_reviewed', 'last_reviewed']] = review_dates[['first_reviewed', 'last_reviewed']].apply(
    pd.to_datetime
)
review_dates[['first_reviewed', 'last_reviewed']] = review_dates[['first_reviewed', 'last_reviewed']].apply(
    lambda x: x.dt.strftime('%Y-%m-%d')
)
review_dates

,first_reviewed,last_reviewed,nb_private_rooms,avg_price
0,2019-01-01,2019-07-09,11356,141.78
